In [ ]:
%run ./Muehle_Utilities.ipynb

## Testboards

In [ ]:
#Startboard
board1 = [[9, 9],
         [[0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0]]]
#Phase 1
board2 = [[7, 6],
         [[0, 0, 0, 0, 2, 0, 0, 0],
          [0, 1, 2, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 1, 2, 0, 0]]]
#Phase 2, keine Mühle
board3 = [[0, 0],
         [[1, 0, 1, 2, 0, 1, 0, 0],
          [2, 0, 1, 0, 0, 0, 1, 0],
          [0, 2, 0, 0, 1, 2, 0, 0]]]
#Phase 2, Mühle
board4 = [[0, 0],
         [[1, 1, 1, 2, 0, 1, 0, 0],
          [2, 0, 1, 0, 0, 0, 1, 0],
          [0, 2, 0, 0, 1, 2, 0, 0]]]
#Phase 2, Spieler 2 kann nicht mehr ziehen
board5 = [[0, 0],
         [[2, 1, 2, 1, 0, 0, 0, 1],
          [2, 2, 1, 0, 0, 1, 2, 1],
          [0, 1, 0, 0, 0, 0, 0, 0]]]
#Phase 3 für Spieler 1
board6 = [[0, 0],
         [[0, 0, 1, 2, 0, 1, 0, 0],
          [2, 0, 0, 0, 0, 0, 0, 0],
          [0, 2, 0, 0, 1, 2, 0, 0]]]
#Spieler 1 hat weniger als 3 Steine
board7 = [[0, 0],
         [[0, 0, 1, 2, 0, 1, 0, 0],
          [2, 0, 0, 0, 0, 0, 0, 0],
          [0, 2, 0, 0, 0, 2, 0, 0]]]

## Spielphasen

Prüft in welcher Phase sich der Spieler befindet

In [ ]:
def get_player_phase(pieces, player):
    [remaining, board] = pieces
    if remaining[player-1] >= 1: # Es wurden noch nicht alle 9 Steine gesetzt
            return 1
    elif count_player_pieces(pieces, player) == 3: # nur noch drei Steine auf dem Spielfeld
        return 3
    else: # sonst
        return 2

Berechnet die nächsten möglichen klickbaren Positionen, falls der Spieler in Phase 1 (Setzphase) ist

In [ ]:
def possible_positions_phase_one(pieces, player):
    return empty_positions(pieces)

Berechnet die nächsten möglichen klickbaren Positionen, falls der Spieler in Phase 2 (Zugphase) ist

In [ ]:
def possible_positions_phase_two(pieces, player, ring, cell):
    collected_piece = (ring, cell)
    neighboring = set(neighboring_positions(collected_piece))
    empty = set(empty_positions(pieces))
    possible_positions = neighboring.intersection(empty)
    #ToDo: Was, wenn kein Stein bewegt werden kann?
    return list(possible_positions)

Berechnet die nächsten möglichen klickbaren Positionen, falls der Spieler in Phase 3 (Springphase) ist

In [ ]:
def possible_positions_phase_three(pieces, player):
    return empty_positions(pieces)

Berechnet in welcher Phase sich der Spieler befindet und dementsprechen, wo er als nächstes klicken kann
Kombiniert also die Funktionen oben

In [ ]:
def next_positions(pieces, player, ring, cell):
    phase = get_player_phase(pieces, player)
    if phase == 1:
        return possible_positions_phase_one(pieces, player)
    elif phase == 2:
        return possible_positions_phase_two(pieces, player, ring, cell)
    else:
        return possible_positions_phase_three(pieces, player)

Prüft, ob das Spiel zu Ende ist und gibt ggf. den Gewinner zurück:
* 0: das Spiel ist noch nicht zu Ende
* 1: Spieler 1 hat gewonnen
* 2: Spieler 2 hat gewonnen

In [ ]:
def finish(pieces):
    [remaining, board] = pieces
    if any(p > 0 for p in remaining): # Wenn man noch Steine setzen kann, hat man nicht verloren
        return 0
    number_pieces = [count_player_pieces(pieces, 1), count_player_pieces(pieces, 2)]
    for player in [1, 2]:
        if number_pieces[player-1] < 3: # Wenn man weniger als 3 Steine hat, hat man verloren
            return opponent(player)
        else:
            # Wenn man in Phase 2 ist und nicht mehr ziehen kann, hat man verloren
            if not any(is_movable(pieces, position) for position in player_pieces(pieces, player)):
                return opponent(player)
    # Falls die beiden Endsituationen für keinen Spieler eingetreten sind, ist das Spiel nicht zu Ende
    return 0

## Spielablauf

In [ ]:
def make_move(ring, cell):
    global status
    phase = get_player_phase(status.pieces, status.current_player)
    possible_positions = next_positions(status.pieces, status.current_player, ring, cell)
    
    if status.number_stones_to_remove == 0:
        if phase == 1:
            if (ring, cell) in possible_positions: # Das geklickte Feld muss frei sein
                status.pieces[1][ring][cell] = status.current_player # Das geklickte Feld wird zum eigenen Feld
                status.pieces[0][status.current_player-1] = status.pieces[0][status.current_player-1]-1 # Die eigenen Steine in remaining um 1 reduzieren
                update_board(status)
                # ToDo: Mühle handeln
            else:
                raise InvalidMoveException()
                print('Du kannst Deinen Stein nur auf ein leeres Feld setzen. Bitte probiere es erneut!')
        elif phase == 2 or phase == 3:
            if status.move_or_jump == False: # -> Stein zum ziehen auswählen
                if status.selected_dot_new in player_pieces(status.pieces, status.current_player): # prüfen,ob der ausgewählte Stein dem Spieler gehört
                    highlight_positions(possible_positions) # erlaubte Zielfelder anzeigen
                    status.move_or_jump = True
                else:
                    raise InvalidMoveException()
                    print('Das ist nicht Dein Stein. Bitte probiere es erneut!')

            elif status.move_or_jump == True: # -> Zielfeld für Stein wählen
                old_ring, old_cell = status.selected_dot_old
                new_ring, new_cell = status.selected_dot_new
                possible_positions = next_positions(status.pieces, status.current_player, old_ring, old_cell) # ausgehend vom ausgewählten Stein, erlaubte Zielfelder berechnen
                if (new_ring, new_cell) in possible_positions: # prüfen ob das ausgewählte Feld ein erlaubtes Zielfeld ist
                    status.pieces[1][new_ring][new_cell] = status.current_player # das Zielfeld als das eigene markieren
                    status.pieces[1][old_ring][old_cell] = 0 # das ursprüngliche Feld ist jetzt leer
                    status.move_or_jump = False
                    update_board(status)
                else:
                    raise InvalidMoveException()
                    print('Du kannst Deinen Stein nicht auf dieses Feld setzen. Bitte probiere es erneut!')
        else:
            raise InvalidMoveException()
            print('Du befindest Dich anscheinend in keiner gültigen Spielphase. Irgendwas ist schief gegangen... Sorry!')
        
        status.number_stones_to_remove = handle_mills(status)
        if status.number_stones_to_remove > 0:
            highlight_positions(get_opponent_beatable_stones(status.current_player), colour = COLOUR_OPPONENT)
    else:
        selected_ring, selected_cell = status.selected_dot_new
        beatable_stones = get_opponent_beatable_stones(status.current_player)
        if (selected_ring, selected_cell) in beatable_stones:
            status.pieces[1][selected_ring][selected_cell] = 0
            status.number_stones_to_remove = status.number_stones_to_remove - 1
            update_board(status)
        else:
            raise InvalidMoveException()
            print('Du kannst diesen Stein nicht entfernen. Bitte probiere es erneut!')
        if status.number_stones_to_remove > 0:
            highlight_positions(get_opponent_beatable_stones(status.current_player), colour = COLOUR_OPPONENT)
    
    if status.move_or_jump == False and status.number_stones_to_remove == 0:
        status.current_player = opponent(status.current_player) # Spielerwechsel, wenn nicht noch eine Aktion ausgeführt werden muss
        
    if finish(status.pieces) != 0: # Prüfen, ob das Spiel zu Ende ist
        status.game_over = True
        winner_popup(finish(status.pieces))

## Mühlen

"handle_mills" gibt die im letzten Zug erzeugten Mühlen für den aktuellen Spieler zurück. Hierzu darf der Spieler erst nach dem Ausführen dieser Funktion wechseln.

In [ ]:
def handle_mills(status):
    pieces = status.pieces
    player = status.current_player
    old_mills = status.mills
    new_mills = set()
    match = set()
    currentPlayerMills = set()

    #find all current mills on the board
    for x in range (0,3):
        for i in [0,2,4,6]:
                if(pieces[1][x][i] == pieces[1][x][i+1] == pieces[1][x][(i+2) % 8] != 0):
                    new_mills.add((pieces[1][x][i], x ,i))

    for i in [1,3,5,7]:
        if(pieces[1][0][i] == pieces[1][1][i] == pieces[1][2][i] != 0):
            new_mills.add((pieces[1][x][i], 0 ,i))

    #set new mills
    status.mills = new_mills   
            

    #if the current mills are the same mills as the old ones or there are now less mills on the board => no new mills
    if(old_mills == new_mills or len(old_mills) > len(new_mills)):
        return(0)


    #find new mills for current player
    if(old_mills != new_mills):
        for mill in new_mills:
            for old_mill in old_mills:
                if mill == old_mill:
                    match.add(mill) 
    new_mills = new_mills.difference(match) #new_mills are now really the new Mills

    #pics mills that belong to the current player 
    for (x,y,z) in new_mills:
        if x == player:
            currentPlayerMills.add((x,y,z))

    return len(currentPlayerMills) #returns count of new mills for the current player

In [ ]:
def get_opponent_beatable_stones(current_player):
    opponent_stones_list = []

    opponent_player = opponent(current_player)
    
    opponent_mill_stones = set()

    #find all opponent_mill_stones | wenn ein Stein in einer Mühle ist, darf er nicht entfernt werden
    _,board = status.pieces

    for ring in range (0,3):
        for cell in [0,2,4,6]:
                if(board[ring][cell] == board[ring][cell+1] == board[ring][(cell+2) % 8] == opponent_player):
                    opponent_mill_stones.add((ring,cell))
                    opponent_mill_stones.add((ring,cell+1))
                    opponent_mill_stones.add((ring,(cell+2) % 8))
    for cell in [1,3,5,7]:
        if(board[0][cell] == board[1][cell] == board[2][cell] == opponent_player):
            opponent_mill_stones.add((0,cell))
            opponent_mill_stones.add((1,cell))
            opponent_mill_stones.add((2,cell))

    for ring in range(0,3):
        for cell in range(0,8):
            if board[ring][cell] == opponent_player:
                if (ring,cell) not in opponent_mill_stones:
                    opponent_stones_list.append((ring,cell)) 
    
    if len(opponent_stones_list) == 0: #falls kein anderer Stein geschlagen werden kann, dürfen auch Steine in Mühlen geschlagen werden
        return list(opponent_mill_stones)
    return opponent_stones_list

## Fehler

Die Exception `InvalidMoveException`, wird später in der Funktion `make_move` geworfen, wenn ein ungültiger Spielzug gefordert wird. Dies dient der Fehlerbehandlung.

In [ ]:
class InvalidMoveException(Exception):
    pass